# Typescript's Type System

## Item 6: Use your editor to interrogate and explorate the type system

When TS is installed, we get two executables:
- `tsc`, the TS compiler
- `tsserver`, the TS standalone server

The server provides server, mostly used by the IDE (formatring, inspection, refactoring...).

Editors can infer types, give hints, refactor... using this server.

## Item 7: Think of types as sets of values

The smallest set is the empty set, which corresponds to the `never` type

In [5]:
let x: never = 12;

1:5 - Type 'number' is not assignable to type 'never'.


The next smallest are the one which contains single values.

In [ ]:
type A = 'A';
type B = 'B';
type Twelve = 12;

Then two or three values

In [7]:
type AB = 'A' | 'B'
type AB12 = 'A' | 'B' | 12

In [8]:
let a: AB12 = 12; // ok
a = 13; // ko

2:1 - Type '13' is not assignable to type 'AB12'.


In [9]:
declare let ab12: AB12;
let b: AB = ab12; // error because AB is NOT a superset of AB12 

2:5 - Type 'AB12' is not assignable to type 'AB'.
2:5 - Type '12' is not assignable to type 'AB'.


As sets, we can do arithmetic operations on them

In [15]:
interface Person {
    name: string;
    age: number;
}

interface Employee {
    grade: string;
}

// just for the example, it makes more sense to extend the first interface
type EmployeePerson = Person & Employee;
let ep: EmployeePerson = {
    name: "truc",
    age: 12,
    grade: "boss"
}

We might speak of `subtypes` when a type is derived from another

In [16]:
interface Vector1D {x: number};
interface Vector2D extends Vector1D {y: number};
interface Vector3D extends Vector2D {z: number};

In a generic type, `extend` also means "subset of" in this context

In [17]:
function getKey<K extends string>(val: any, key: K) {
    // ...
}

In [19]:
getKey({}, 'x'); // 'x' extends string
getKey({}, Math.random() < 0.5 ? 'a' : 'b'); // 'a'|'b' extends string
getKey({}, 12); // NOPE

3:12 - Argument of type 'number' is not assignable to parameter of type 'string'.


In [20]:
function sortBy<K extends keyof T, T>(vals: T[], key: K): T[] {
    return vals // just for example
}

1:59 - A function whose declared type is neither 'void' nor 'any' must return a value.


## Item 8: Know how to tell whether a symbol is in the type space of value space

In [22]:
interface Cylinder {
    radius: number;
    height: number;
}

let Cylinder = (radius: number, height: number): Cylinder => ({radius, height});

The `interface` defines `Cylinder` in the type scope, whereas `Cylinder` function is in the value space.

In [24]:
const v = typeof Cylinder; // Value is "function"
type T = typeof Cylinder; // Type is typeof Cylinder

`obj['field']` and `obj.field` are both defined in value space, but not in type space

In [ ]:
let sendEmail = ({address: string, subject: string, body: string}) => {
    // ...
}

We can't use destructuring assignment in TS, we split types and values.

In [31]:
let sendEmail = (
    {address, subject, body}: {address: string, subject: string, body: string}
) => {
    // ...
}

## Item 9: Prefer type declarations to type assertions

In [37]:
interface Person1 {name: string};

let alice: Person1 = {name: "Alice"}; // type is Person1
let bob = {name: "Bob"} as Person1 // type is Person1

Prefer type declaration (first instruction) as type assertions (second instruction). Type assertion are a kind of "cast" 

In [33]:
let alice2: Person1 = {} // error, which is great
let bob2 = {} as Person1 // no error detected because of type assertion

1:5 - Property 'name' is missing in type '{}' but required in type 'Person1'.


In [35]:
let alice3: Person1 = {
    name: "alice",
    occupation: "developer"
} // error because occupaation is not referenced 
let bob3 = {
    name: "bob",
    occupation: "developer"
} as Person1

3:5 - Type '{ name: string; occupation: string; }' is not assignable to type 'Person1'.
3:5 - Object literal may only specify known properties, and 'occupation' does not exist in type 'Person1'.


Type assertion makes sense when you have more information than Typescript (e.g. when fetching a DOM element)

Used as a suffix, `!` is interpreted as an assertion that the value is non-null.

Using `unknown` makes every assertion possible (because every type is a subset of `unknown`), even the most suspicious ones

In [47]:
const pierre = {fullname: "pierre"} as Person1 // error
const marie = {fullname: "marie"} as unknown as Person1

1:16 - Conversion of type '{ fullname: string; }' to type 'Person1' may be a mistake because neither type sufficiently overlaps with the other. If this was intentional, convert the expression to 'unknown' first.
1:16 - Property 'name' is missing in type '{ fullname: string; }' but required in type 'Person1'.


Unlike `any`, you cannot access any properties of `unknown`

In [50]:
let tom = {name: "tom"} as unknown
tom.name // KO
let tom2 = tom as Person1
tom2.name // OK

2:5 - Property 'name' does not exist on type 'unknown'.


## Item 10: Avoid object wrapper types (String, Number, Boolean, Symbol, BigInt)

Don't use type wrappers (e.g. `String` instead of real primitive type `string`). If you still want, it's OK to call them without `new`

In [51]:
"a" === new String('a')

false


In [53]:
"a" === String('a')

true


In [56]:
typeof String("a")

string


In [57]:
typeof new String("a")

object


## Item 11: Recognize the limits of excess property checking

In [67]:
interface Room {
    numDoors: number;
    ceilingHeightFt: number;
}

In [69]:
const r: Room = {
    numDoors: 1,
    ceilingHeightFt: 10,
    elephant: 'present',   
};

4:5 - Type '{ numDoors: number; ceilingHeightFt: number; elephant: string; }' is not assignable to type 'Room'.
4:5 - Object literal may only specify known properties, and 'elephant' does not exist in type 'Room'.


But this will work with assignation

In [73]:
const obj = {
    numDoors: 1,
    ceilingHeightFt: 10,
    elephant: 'present'
}

const room: Room = obj;

In the first case, we have triggered a process knwon as "excess property checking", but not in the second one

## Item 12: Apply types to entire function expressions when possible

In [74]:
function add(a: number, b: number) { return a + b; }
function sub(a: number, b: number) { return a - b; }
function mul(a: number, b: number) { return a * b; }
function div(a: number, b: number) { return a / b; }

To avoid annotations, we can add types to the entire function

In [76]:
type BinaryFn = (a: number, b: number) => number;
const simpleAdd: BinaryFn = (a, b) => a + b;
const simpleSub: BinaryFn = (a, b) => a - b;
const simpleMul: BinaryFn = (a, b) => a * b;
const simpleDiv: BinaryFn = (a, b) => a / b;

Also, for async functions based on fetch

In [ ]:
declare function fetch(
    input: RequestInfo, init?: RequestInit
): Promise<Response>;

In [ ]:
async function checkedFetch(input: RequestInfo, init?: RequestInit) {
    const response = await fetch(input, init);
    if (!response.ok) {
        // Converted to a rejected Promise in an async function
        throw new Error('Request failed: ' + response.status);
    }
    return response;
}

can be turned into

In [ ]:
const checkedFetch: typeof fetch = async (input, init) => {
    const response = await fetch(input, init);
    if (!response.ok) {
        throw new Error('Request failed: ' + response.status);
    }
    return response;
}

## Item 13: Know the differences between type and interface

Pretty much the same for a lot of situations.

You can't union interfaces

You can augment an interface, it's called "declaration merging" 

In [2]:
interface IState {
    name: string;
    capital: string;
}
interface IState {
    population: number;
}
const wyoming: IState = {
    name: 'Wyoming',
    capital: 'Cheyenne',
    population: 500_000
}; // OK

## Item 14: Use type operations and generics to avoid repeating yourself

Use more `&` and `|` operations to concat / intersect types

Use `Partial`, `Pick`, `ReturnType` utility types